In [28]:
import sys
sys.path.append("../")
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
import numpy as np
import random
import time 
from core.data_handler import *
from core.inference_handler import *

# Simple code to generate a new map
def cleanMap(center, boundary): 
    # Mark transformer
    tMarker =  Marker(location=center)
    # Mark polygon
    tPoly = Polygon(locations=boundary, weight=2,
                color='#003d99', opacity=0.4, fill_opacity=0.0,
                fill_color='#ccffcc') 
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=center ,zoom=16)
    #tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

%load_ext autoreload
%autoreload 2

def cleanMap(center, boundary): 
    # Mark transformer
    tMarker =  Marker(location=center)
    # Mark polygon
    tPoly = Polygon(locations=boundary, weight=3,
                color='#003d99', opacity=0.8, fill_opacity=0.0,
                fill_color='#ccffcc') 
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=center ,zoom=16)
    tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Connect to db 

In [30]:
conn = connect_to_psql(dbname='gis', user='jennyzhou')

Connection established. DNS info: dbname=gis user=jennyzhou


## Define the Freimann subarea

In [10]:
# Define freimann subarea 
area_id = 3
core_center, core_boundary = select_freimann_subarea(area_id=area_id, conn=conn)
# tMap = cleanMap(core_center, core_boundary)
# tMap

Select freimann subarea with id=3


## Generate inference step by step

### Step0. Use roads from osm and split them by road crosses

In [16]:
# Select roads within the area
roads = select_roads(area_id, conn)

# Split roads by road intersections
split_roads = split_by_difference(roads)
print 'Original roads: %d Split roads: %d'%(len(roads), len(split_roads))

# Uncomment the following lines to visualize the split roads
# tMap = cleanMap(core_center, core_boundary)
# for line in split_roads:
#     color = "#%06x" % random.randint(0, 0xFFFFFF)
#     pl = Polyline(locations=line, weight=2, color=color, opacity=0.9)
#     tMap.add_layer(pl)
    
# tMap

Original roads: 21 Split roads: 53


### Step1. Choose road end points within the subarea as road nodes

In [17]:
# Find road nodes
road_nodes = set()
for rd in split_roads:
    road_nodes.add(tuple(rd[0]))
    road_nodes.add(tuple(rd[-1]))
print 'Approximated road nodes %d' % len(road_nodes)

# Filter out road nodes out of the area boundary
filtered_road_nodes = filter_by_boundary(road_nodes, core_boundary)

# Uncomment the following lines to visualize the road nodes within the boundary
# tMap = cleanMap(core_center, core_boundary)
# for nd in filtered_road_nodes:
#     omk = Circle(location=nd, weight=1, opacity = 0.8, color = 'green', radius = 1) 
#     tMap.add_layer(omk)
# tMap

Approximated road nodes 37
Original nodes 37 >> Strictly within area 25


### Step2. Prepare nodes and edges for graph inference

In [20]:
# Find out all the target nodes in table freimann_processed_nodes and their projection onto roads
orig_nodes, proj_nodes, map_edges = select_nodes_and_projection(area_id, conn)
print 'Original nodes %d Proj nodes %d' % (len(orig_nodes), len(proj_nodes))

# Uncomment the following lines to visualize the target nodes and their projection within the boundary
# tMap = cleanMap(core_center, core_boundary)
# for nd in orig_nodes:
#     omk = Circle(location=nd, weight=1, opacity = 0.8, color = 'blue', radius = 1) 
#     tMap.add_layer(omk)

# for nd in proj_nodes:
#     omk = Circle(location=nd, weight=1, opacity = 0.8, color = 'green', radius = 1) 
#     tMap.add_layer(omk)

# for line in map_edges:
#     pl = Polyline(locations=line, weight=1, color='red', opacity=0.7)
#     tMap.add_layer(pl)
# tMap

Original nodes 169 Proj nodes 169


### Step3. Generate inference graph by finding a minimum spanning tree 
### MST Version 1 with direct connection to house

In [22]:
# V1: Generate MST with all nodes
MST1 = generate_mst_direct(list(filtered_road_nodes | orig_nodes | proj_nodes))
sav_graph_npy('../graph/aid_%d_inf_v1.npy'%area_id, MST1)

# Uncomment the following lines to visualize graph
# tMap = cleanMap(core_center, core_boundary)
# for tp in MST1.nodes():
#     cs = Circle(location=tp, weight=1, opacity = 0.9, color = 'yellow', radius = 1) 
#     tMap.add_layer(cs)

# for line in MST1.edges():
#     pl = Polyline(locations=line, weight=1, color='blue', opacity=0.7)
#     tMap.add_layer(pl)
# tMap

Graph edges 362, nodes 363
Save graph as ../graph/aid_3_inf_v1.npy


### MST Version 2 manually added connection to house

In [25]:
# V2: Generate MST with projected nodes only
MST2 = generate_mst_improved(list(filtered_road_nodes | proj_nodes), map_edges)
sav_graph_npy('../graph/aid_%d_inf_v2.npy'%area_id, MST2)

# Uncomment the following lines to visualize graph
# tMap = cleanMap(core_center, core_boundary)
# for tp in MST2.nodes():
#     cs = Circle(location=tp, weight=1, opacity = 0.9, color = 'yellow', radius = 1) 
#     tMap.add_layer(cs)

# for line in MST2.edges():
#     pl = Polyline(locations=line, weight=1, color='blue', opacity=0.7)
#     tMap.add_layer(pl)
# tMap

Graph edges 193, nodes 194
After adding house connection >> edges 362, nodes 363
Save graph as ../graph/aid_3_inf_v2.npy


## Load saved graph and save as csv for visualization in QGIS

In [26]:
# MST V1
G1, total_length = load_graph_npy('../graph/aid_%d_inf_v1.npy'%area_id)
print 'Graph edge length: %s'%total_length
# Save G1 as csv file
sav_graph_to_csv(save_dir='../graph', prefix='aid_%d_inf_v1'%area_id, graph=G1) 

# MST V2
G2, total_length = load_graph_npy('../graph/aid_%d_inf_v2.npy'%area_id)
print 'Graph edge length: %s'%total_length
# Save G2 as csv file
sav_graph_to_csv(save_dir='../graph', prefix='aid_%d_inf_v2'%area_id, graph=G2) 

Graph edge length: 3025.46761864
Save to ../graph/aid_3_inf_v1_nodes.csv
Save to ../graph/aid_3_inf_v1_edges.csv
Graph edge length: 3342.53162072
Save to ../graph/aid_3_inf_v2_nodes.csv
Save to ../graph/aid_3_inf_v2_edges.csv
